<a href="https://colab.research.google.com/github/shinoyom89-bit/ml-scikit/blob/main/MAGIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cols = ["flength","fwidth","fsize","fconc","fconc1","fasym","fn3long","fm3trans","falpha","fdist","class"]
df = pd.read_csv('/content/magic04.data', names=cols)
df.head()


,flength,fwidth,fsize,fconc,fconc1,fasym,fn3long,fm3trans,falpha,fdist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [ ]:
df['class']=(df['class']=="g").astype(int) # here g is taken as int and h as str so g becoma one and h is zero

In [ ]:
for label in cols[:-1]:
    plt.hist(df[df["class"]==1][label],color="blue",label="gamma",alpha=0.7,density=True)
    plt.hist(df[df["class"]==0][label],color="red",label="hydron",alpha=0.7,density=True)
    plt.title(label)#acha to ye bins mai dicivide karde and y mai usme
    # kitni elemnt ya number uske ocunts dega
    plt.xlabel(label)
    plt.ylabel("probablity")
    plt.legend()
    plt.show()

TRAIN ,TESTS ,VALIDATION DATA SETS HERE

In [ ]:
shuffled = df.sample(frac=1) # shuffel data set here
train = shuffled.iloc[:int(0.6*len(df))] #60%
valid = shuffled.iloc[int(0.6*len(df)):int(0.8*len(df))]#20%
test  = shuffled.iloc[int(0.8*len(df)):]#20%

In [ ]:
def scale_dataset(dataframe,oversample=False):
    x=dataframe[cols[:-1]].values
    y=dataframe[cols[-1]].values #Original y is 1D, like a single row of numbers: [0, 1, 0, 1]
#y.reshape(-1,1) → converts it into a column (vertical)
    scaler=StandardScaler()
    x=scaler.fit_transform(x)
    if oversample:
        ros=RandomOverSampler()
        x,y=ros.fit_resample(x,y) # resize the smaller class
    data = np.hstack((x,y.reshape(-1,1))) # Reshape y to be a column vector before stacking
    return data,x,y # the first array in the output is of data and second array is of the x and third is of y

In [ ]:
train,x_train,y_train=scale_dataset(train,oversample=True) # here we store the data and x and y in the train and x_train and ytrain
valid,x_valid,y_valid=scale_dataset(valid,oversample=False) # we set this to flase to take it as unseen data for our model
test,x_test,y_test=scale_dataset(test,oversample=False) # we take the false here to take it as unseen data for the test

In [ ]:
(y_train == 1).sum()# here y is a series so we just pass where y is 1

np.int64(7373)

In [ ]:
(y_train == 0).sum()

np.int64(7373)

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
knn_model=KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x_train,y_train) # give the model training data
# To 1st argument = input data
# 2nd argument = us input ka actual output (jo model ko seekhna hai)

KNeighborsClassifier(n_neighbors=1)

In [ ]:
y_prid=knn_model.predict(x_test) # the  y is the target and x we give to model to get train and find the nearest elemnt usin vector distance
# or the prediction of the target on the training data by using the test data
y_prid

array([1, 1, 1, ..., 0, 1, 1])

In [ ]:
y_test

array([0, 1, 0, ..., 1, 1, 1])

In [ ]:
print(classification_report(y_test,y_prid)) # cheking the real target from the test and comparing it to the predicted one
# To 1st argument = input data
# 2nd argument = us input ka actual output (jo model ko seekhna hai)

              precision    recall  f1-score   support

           0       0.75      0.69      0.72      1285
           1       0.85      0.88      0.87      2519

    accuracy                           0.82      3804
   macro avg       0.80      0.79      0.79      3804
weighted avg       0.82      0.82      0.82      3804

